In [1]:
import pandas
import networkx as nx
import pickle

In [3]:
def create_graph_object(num, name):
    if name == "fully":
        G = nx.complete_graph(num)
    elif name == "star":
        G = nx.star_graph(num - 1)
    elif name == "ring":
        G = nx.cycle_graph(num)
    elif name.startswith("ER_"):
        try:
            # Extract probability and seed from the graph_info
            parts = name.split("_")
            prob = float(parts[1])
            G = nx.erdos_renyi_graph(num, prob, seed=45)
        except (IndexError, ValueError):
            raise ValueError("Invalid format for ER graph type. Expected 'ER_prob'")
    else:
        raise ValueError("Not supported topology.")        
    return G

In [4]:
for num in [10,20,30]:
    for name in ["fully", "star", "ring", "ER_0.3", "ER_0.7", "ER_0.5"]:
        G = create_graph_object(num, name)
        with open(f'{num}_{name}.pk','wb') as f:
            pickle.dump(G, f)

In [5]:
for file_name in ['Abilene_edges.csv', 'GÉANT_edges.csv', 'synth50_edges.csv', 'rf1755_edges.csv', 'rf3967_edges.csv']:
    edges_df = pandas.read_csv(file_name)
    parts = file_name.split("_")
    topo = parts[0]
    G = nx.DiGraph()
    for _, row in edges_df.iterrows():
        G.add_edge(row['src'], row['dst'])
    adj_matrix = nx.adjacency_matrix(G).todense()
    print(file_name, len(adj_matrix))
    with open(f'{len(adj_matrix)}_{topo}.pk','wb') as f:
        pickle.dump(G, f)

Abilene_edges.csv 12
GÉANT_edges.csv 22
synth50_edges.csv 50
rf1755_edges.csv 87
rf3967_edges.csv 79


In [2]:
import requests
from bs4 import BeautifulSoup
import os
import time
import xml.etree.ElementTree as ET
import networkx as nx
import pandas as pd
import pickle

In [4]:
NETWORK_LIST = [
    "abilene", "atlanta", "brain", "cost266", "dfn-bwin", "dfn-gwin", "di-yuan",
    "france", "geant", "germany50", "giul39", "india35", "janos-us", "janos-us-ca",
    "newyork", "nobel-eu", "nobel-germany", "nobel-us", "norway", "pdh", "pioro40",
    "polska", "sun", "ta1", "ta2", "zib54"
]

BASE_URL = "https://sndlib.put.poznan.pl/download/sndlib-networks-xml/"

def find_tag_ignore_namespace(root, tag):
    return next((elem for elem in root.iter() if elem.tag.endswith(tag)), None)

def find_all_tags_ignore_namespace(root, tag):
    return [elem for elem in root.iter() if elem.tag.endswith(tag)]

def safe_find_text(elem, tag):
    for child in elem:
        if child.tag.endswith(tag):
            return child.text
    return None

def parse_and_save_adj_matrix(name, save_dir="adj_matrices"):
    os.makedirs(save_dir, exist_ok=True)
    url = f"{BASE_URL}{name}.xml"
    # print(f"Fetching {url}")
    try:
        r = requests.get(url)
        r.raise_for_status()
        root = ET.fromstring(r.content)

        G = nx.DiGraph(name=name)

        node_elements = find_all_tags_ignore_namespace(root, 'node')
        link_elements = find_all_tags_ignore_namespace(root, 'link')

        if not node_elements or not link_elements:
            raise ValueError("Missing <node> or <link> in XML")

        # Map node IDs to 0-based indices
        node_id_map = {}
        # print(len(node_elements))
        # print(len(link_elements))
        for idx, node in enumerate(node_elements):
            node_id_map[node.attrib['id']] = idx
            G.add_node(idx)

        # Add edges using 0-based node indices
        for link in link_elements:
            src = safe_find_text(link, 'source')
            dst = safe_find_text(link, 'target')
            if src is not None and dst is not None:
                G.add_edge(node_id_map[src], node_id_map[dst])
        # Save adjacency matrix
        adj_matrix = nx.adjacency_matrix(G).todense()
        
        print(name, len(adj_matrix))
        # with open(f'{len(adj_matrix)}_{name}.pk','wb') as f:
        #     pickle.dump(G, f)
        return len(adj_matrix)

    except Exception as e:
        print(f"Failed to process {name}: {e}")

def process_all_networks():
    nodes = []
    topo = []
    for name in NETWORK_LIST:
        topo.append(name)
        num = parse_and_save_adj_matrix(name)
        nodes.append(num)
        time.sleep(0.5)  # polite delay
    print(topo)
    print(nodes)

if __name__ == "__main__":
    process_all_networks()

abilene 12
atlanta 15
brain 161
cost266 37
dfn-bwin 10
dfn-gwin 11
di-yuan 11
france 25
geant 22
germany50 50
giul39 39
india35 35
janos-us 26
janos-us-ca 39
newyork 16
nobel-eu 28
nobel-germany 17
nobel-us 14
norway 27
pdh 11
pioro40 40
polska 12
sun 27
ta1 24
ta2 65
zib54 54
['abilene', 'atlanta', 'brain', 'cost266', 'dfn-bwin', 'dfn-gwin', 'di-yuan', 'france', 'geant', 'germany50', 'giul39', 'india35', 'janos-us', 'janos-us-ca', 'newyork', 'nobel-eu', 'nobel-germany', 'nobel-us', 'norway', 'pdh', 'pioro40', 'polska', 'sun', 'ta1', 'ta2', 'zib54']
[12, 15, 161, 37, 10, 11, 11, 25, 22, 50, 39, 35, 26, 39, 16, 28, 17, 14, 27, 11, 40, 12, 27, 24, 65, 54]


In [5]:
import os
import pickle as pk

In [6]:
all_topofiles = os.listdir()
infos = []
for topof in all_topofiles:
    if '.pk' in topof:
        with open(topof, "rb") as f:
            G = pk.load(f)
            num_nodes = G.number_of_nodes()
            num_edges = G.number_of_edges()
            avg_degree = sum(dict(G.degree()).values()) / num_nodes
            density = nx.density(G)
            g_info = [topof, num_nodes, num_edges,avg_degree ,density]
            infos.append(g_info)
            

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(infos, columns=['topo', 'num_nodes', 'num_edges','avg_degree' ,'density'])

In [11]:
df.to_csv('topo.csv')